1. Write Data to Delta Lake (managed table)
2. Write Data to Delta Lake (external table)
3. Read data from Delta Lake (table)
4. Read data from Delta Lake (file)

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS f1_demo
LOCATION '/mnt/formula1dl612/demo'

In [0]:

results_df = spark.read\
.option("inferSchema", True)\
.json("/mnt/formula1dl612/raw/2021-03-28/results.json")

In [0]:
results_df.write.format("delta").mode("overwrite").saveAsTable("f1_demo.results_managed")

In [0]:
%sql
SELECT * FROM f1_demo.results_managed;

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,25,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,18,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,16,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,12,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,10,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,8,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,6,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,4,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,2,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,1,10,10,10,1052,9,24975,1,+86.713


In [0]:
results_df.write.format("delta").mode("overwrite").save("/mnt/formula1dl612/demo/results_external")

In [0]:
%sql
CREATE TABLE f1_demo.results_external
USING DELTA
LOCATION '/mnt/formula1dl612/demo/results_external'

In [0]:
%sql
SELECT * FROM f1_demo.results_external

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,25,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,18,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,16,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,12,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,10,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,8,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,6,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,4,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,2,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,1,10,10,10,1052,9,24975,1,+86.713


In [0]:
results_external_df = spark.read.format("delta").load("/mnt/formula1dl612/demo/results_external")

In [0]:
display(results_external_df)

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,25,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,18,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,16,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,12,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,10,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,8,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,6,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,4,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,2,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,1,10,10,10,1052,9,24975,1,+86.713


In [0]:
results_df.write.format("delta").mode("overwrite").partitionBy('constructorId').saveAsTable("f1_demo.results_partitioned")

In [0]:
%sql
SHOW PARTITIONS f1_demo.results_partitioned

constructorId
1
3
6
9
51
131
117
214
210
213


1. Update Delta Table
2. Delete From DELTA Table

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,25,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,18,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,16,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,12,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,10,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,8,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,6,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,4,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,2,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,1,10,10,10,1052,9,24975,1,+86.713


In [0]:
%sql
UPDATE f1_demo.results_managed
SET points = 11 - position
WHERE position <= 10

num_affected_rows
10


In [0]:
%sql
SELECT * FROM f1_demo.results_managed

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,10,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,9,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,8,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,7,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,6,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,5,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,4,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,3,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,2,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,1,10,10,10,1052,9,24975,1,+86.713


In [0]:
from delta.tables import DeltaTable

In [0]:
deltaTable = DeltaTable.forPath(spark, "/mnt/formula1dl612/demo/results_managed")

deltaTable.update("position <= 10", { "points" : "21 - position"})

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,20,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,19,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,18,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,17,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,16,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,15,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,14,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,13,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,12,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,11,10,10,10,1052,9,24975,1,+86.713


In [0]:
%sql
DELETE FROM f1_demo.results_managed
WHERE position > 10;

num_affected_rows
8


In [0]:
deltaTable = DeltaTable.forPath(spark, "/mnt/formula1dl612/demo/results_managed")

deltaTable.delete("points = 0")

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,20,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,19,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,18,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,17,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,16,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,15,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,14,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,13,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,12,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,11,10,10,10,1052,9,24975,1,+86.713


https://docs.delta.io/latest/delta-update.html#-delta-merge&language-sql

#### Upsert using merge

In [0]:
drivers_day1_df = spark.read\
.option("inferSchema", True)\
.json("/mnt/formula1dl612/raw/2021-03-28/drivers.json")\
.filter("driverId <= 10")\
.select("driverId", "dob", "name.forename", "name.surname")   

In [0]:
display(drivers_day1_df)

driverId,dob,forename,surname
1,1985-01-07,Lewis,Hamilton
2,1977-05-10,Nick,Heidfeld
3,1985-06-27,Nico,Rosberg
4,1981-07-29,Fernando,Alonso
5,1981-10-19,Heikki,Kovalainen
6,1985-01-11,Kazuki,Nakajima
7,1979-02-28,Sébastien,Bourdais
8,1979-10-17,Kimi,Räikkönen
9,1984-12-07,Robert,Kubica
10,1982-03-18,Timo,Glock


In [0]:
drivers_day1_df.createOrReplaceTempView("drivers_day1")

In [0]:
from pyspark.sql.functions import upper
drivers_day2_df = spark.read\
.option("inferSchema", True)\
.json("/mnt/formula1dl612/raw/2021-03-28/drivers.json")\
.filter("driverId BETWEEN 6 AND 15")\
.select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))
display(drivers_day2_df)

driverId,dob,forename,surname
6,1985-01-11,KAZUKI,NAKAJIMA
7,1979-02-28,SÉBASTIEN,BOURDAIS
8,1979-10-17,KIMI,RÄIKKÖNEN
9,1984-12-07,ROBERT,KUBICA
10,1982-03-18,TIMO,GLOCK
11,1977-01-28,TAKUMA,SATO
12,1985-07-25,NELSON,PIQUET JR.
13,1981-04-25,FELIPE,MASSA
14,1971-03-27,DAVID,COULTHARD
15,1974-07-13,JARNO,TRULLI


In [0]:
drivers_day2_df.createOrReplaceTempView("drivers_day2")

In [0]:
drivers_day3_df = spark.read\
.option("inferSchema", True)\
.json("/mnt/formula1dl612/raw/2021-03-28/drivers.json")\
.filter("driverId BETWEEN 1 AND 5 OR driverId BETWEEN 16 AND 20")\
.select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))
display(drivers_day3_df)

driverId,dob,forename,surname
1,1985-01-07,LEWIS,HAMILTON
2,1977-05-10,NICK,HEIDFELD
3,1985-06-27,NICO,ROSBERG
4,1981-07-29,FERNANDO,ALONSO
5,1981-10-19,HEIKKI,KOVALAINEN
16,1983-01-11,ADRIAN,SUTIL
17,1976-08-27,MARK,WEBBER
18,1980-01-19,JENSON,BUTTON
19,1979-04-18,ANTHONY,DAVIDSON
20,1987-07-03,SEBASTIAN,VETTEL


In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_merge (
driverID INT,
dob DATE,
forename STRING,
surname STRING,
createdDate DATE,
updatedDate DATE
)
USING DELTA

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day1 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN
  UPDATE SET  tgt.dob = upd.dob,
              tgt.forename = upd.forename,
              tgt.surname = upd.surname,
              tgt.updatedDate = current_timestamp
WHEN NOT MATCHED
  THEN INSERT (driverId, dob, forename, surname, createdDate) VALUES (driverID, dob, forename, surname, current_timestamp)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,0,0,10


In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge;

driverID,dob,forename,surname,createdDate,updatedDate
1,1985-01-07,Lewis,Hamilton,2024-03-16,null
2,1977-05-10,Nick,Heidfeld,2024-03-16,null
3,1985-06-27,Nico,Rosberg,2024-03-16,null
4,1981-07-29,Fernando,Alonso,2024-03-16,null
5,1981-10-19,Heikki,Kovalainen,2024-03-16,null
6,1985-01-11,Kazuki,Nakajima,2024-03-16,null
7,1979-02-28,Sébastien,Bourdais,2024-03-16,null
8,1979-10-17,Kimi,Räikkönen,2024-03-16,null
9,1984-12-07,Robert,Kubica,2024-03-16,null
10,1982-03-18,Timo,Glock,2024-03-16,null


In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day2 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN
  UPDATE SET  tgt.dob = upd.dob,
              tgt.forename = upd.forename,
              tgt.surname = upd.surname,
              tgt.updatedDate = current_timestamp
WHEN NOT MATCHED
  THEN INSERT (driverId, dob, forename, surname, createdDate) VALUES (driverID, dob, forename, surname, current_timestamp)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,5,0,5


In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge;

driverID,dob,forename,surname,createdDate,updatedDate
6,1985-01-11,KAZUKI,NAKAJIMA,2024-03-16,2024-03-16
7,1979-02-28,SÉBASTIEN,BOURDAIS,2024-03-16,2024-03-16
8,1979-10-17,KIMI,RÄIKKÖNEN,2024-03-16,2024-03-16
9,1984-12-07,ROBERT,KUBICA,2024-03-16,2024-03-16
10,1982-03-18,TIMO,GLOCK,2024-03-16,2024-03-16
11,1977-01-28,TAKUMA,SATO,2024-03-16,null
12,1985-07-25,NELSON,PIQUET JR.,2024-03-16,null
13,1981-04-25,FELIPE,MASSA,2024-03-16,null
14,1971-03-27,DAVID,COULTHARD,2024-03-16,null
15,1974-07-13,JARNO,TRULLI,2024-03-16,null


In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import current_timestamp

deltaTable = DeltaTable.forPath(spark, "/mnt/formula1dl612/demo/drivers_merge")


deltaTable.alias('tgt') \
  .merge(
    drivers_day3_df.alias('upd'),
    "tgt.driverId = upd.driverId"
  ) \
  .whenMatchedUpdate(set =
    {
      "dob": "upd.dob",
      "forename": "upd.forename",
      "surname": "upd.surname",
      "updatedDate": "current_timestamp()"
    }
  ) \
  .whenNotMatchedInsert(values =
    {
    
      "driverId": "upd.driverId",
      "dob": "upd.dob",
      "forename": "upd.forename",
      "surname": "upd.surname",
      "createdDate": "current_timestamp()"
    }
  ) \
  .execute()

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge;

driverID,dob,forename,surname,createdDate,updatedDate
6,1985-01-11,KAZUKI,NAKAJIMA,2024-03-16,2024-03-16
7,1979-02-28,SÉBASTIEN,BOURDAIS,2024-03-16,2024-03-16
8,1979-10-17,KIMI,RÄIKKÖNEN,2024-03-16,2024-03-16
9,1984-12-07,ROBERT,KUBICA,2024-03-16,2024-03-16
10,1982-03-18,TIMO,GLOCK,2024-03-16,2024-03-16
11,1977-01-28,TAKUMA,SATO,2024-03-16,null
12,1985-07-25,NELSON,PIQUET JR.,2024-03-16,null
13,1981-04-25,FELIPE,MASSA,2024-03-16,null
14,1971-03-27,DAVID,COULTHARD,2024-03-16,null
15,1974-07-13,JARNO,TRULLI,2024-03-16,null


1. History & Versioning
2. Time Travel
3. Vaccum (Good for GDPR solutions)

In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2024-03-16T23:52:27.000+0000,2818554889652103,gilbaram20@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#16346 as bigint) = driverId#16299L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(4462865897687888),0124-093932-co5lwa5n,2,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 2551, numTargetRowsInserted -> 5, scanTimeMs -> 1199, numTargetRowsUpdated -> 5, numOutputRows -> 10, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 1, rewriteTimeMs -> 1030)",null,Databricks-Runtime/11.3.x-scala2.12
2,2024-03-16T23:31:48.000+0000,2818554889652103,gilbaram20@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#15045 as bigint) = driverId#14971L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(4462865897687888),0124-093932-co5lwa5n,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 5, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, executionTimeMs -> 2990, numTargetRowsInserted -> 5, scanTimeMs -> 1502, numTargetRowsUpdated -> 5, numOutputRows -> 15, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 1, rewriteTimeMs -> 972)",null,Databricks-Runtime/11.3.x-scala2.12
1,2024-03-16T23:27:41.000+0000,2818554889652103,gilbaram20@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#13823 as bigint) = driverId#12836L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(4462865897687888),0124-093932-co5lwa5n,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 2680, numTargetRowsInserted -> 10, scanTimeMs -> 1524, numTargetRowsUpdated -> 0, numOutputRows -> 10, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 0, rewriteTimeMs -> 653)",null,Databricks-Runtime/11.3.x-scala2.12
0,2024-03-16T23:25:43.000+0000,2818554889652103,gilbaram20@outlook.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(4462865897687888),0124-093932-co5lwa5n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/11.3.x-scala2.12


In [0]:
%sql
SELECT * from f1_demo.drivers_merge Version AS OF 1;

driverID,dob,forename,surname,createdDate,updatedDate
1,1985-01-07,Lewis,Hamilton,2024-03-16,null
2,1977-05-10,Nick,Heidfeld,2024-03-16,null
3,1985-06-27,Nico,Rosberg,2024-03-16,null
4,1981-07-29,Fernando,Alonso,2024-03-16,null
5,1981-10-19,Heikki,Kovalainen,2024-03-16,null
6,1985-01-11,Kazuki,Nakajima,2024-03-16,null
7,1979-02-28,Sébastien,Bourdais,2024-03-16,null
8,1979-10-17,Kimi,Räikkönen,2024-03-16,null
9,1984-12-07,Robert,Kubica,2024-03-16,null
10,1982-03-18,Timo,Glock,2024-03-16,null


In [0]:
%sql
SELECT * from f1_demo.drivers_merge Version AS OF 2;

driverID,dob,forename,surname,createdDate,updatedDate
6,1985-01-11,KAZUKI,NAKAJIMA,2024-03-16,2024-03-16
7,1979-02-28,SÉBASTIEN,BOURDAIS,2024-03-16,2024-03-16
8,1979-10-17,KIMI,RÄIKKÖNEN,2024-03-16,2024-03-16
9,1984-12-07,ROBERT,KUBICA,2024-03-16,2024-03-16
10,1982-03-18,TIMO,GLOCK,2024-03-16,2024-03-16
11,1977-01-28,TAKUMA,SATO,2024-03-16,null
12,1985-07-25,NELSON,PIQUET JR.,2024-03-16,null
13,1981-04-25,FELIPE,MASSA,2024-03-16,null
14,1971-03-27,DAVID,COULTHARD,2024-03-16,null
15,1974-07-13,JARNO,TRULLI,2024-03-16,null


In [0]:
%sql
SELECT * from f1_demo.drivers_merge TIMESTAMP AS OF '2024-03-16T23:27:41.000+0000';

driverID,dob,forename,surname,createdDate,updatedDate
1,1985-01-07,Lewis,Hamilton,2024-03-16,null
2,1977-05-10,Nick,Heidfeld,2024-03-16,null
3,1985-06-27,Nico,Rosberg,2024-03-16,null
4,1981-07-29,Fernando,Alonso,2024-03-16,null
5,1981-10-19,Heikki,Kovalainen,2024-03-16,null
6,1985-01-11,Kazuki,Nakajima,2024-03-16,null
7,1979-02-28,Sébastien,Bourdais,2024-03-16,null
8,1979-10-17,Kimi,Räikkönen,2024-03-16,null
9,1984-12-07,Robert,Kubica,2024-03-16,null
10,1982-03-18,Timo,Glock,2024-03-16,null


In [0]:
df = spark.read.format("delta").option("timestampAsOf", '2024-03-16T23:27:41.000+0000').load("/mnt/formula1dl612/demo/drivers_merge")

In [0]:
display(df)

driverID,dob,forename,surname,createdDate,updatedDate
1,1985-01-07,Lewis,Hamilton,2024-03-16,null
2,1977-05-10,Nick,Heidfeld,2024-03-16,null
3,1985-06-27,Nico,Rosberg,2024-03-16,null
4,1981-07-29,Fernando,Alonso,2024-03-16,null
5,1981-10-19,Heikki,Kovalainen,2024-03-16,null
6,1985-01-11,Kazuki,Nakajima,2024-03-16,null
7,1979-02-28,Sébastien,Bourdais,2024-03-16,null
8,1979-10-17,Kimi,Räikkönen,2024-03-16,null
9,1984-12-07,Robert,Kubica,2024-03-16,null
10,1982-03-18,Timo,Glock,2024-03-16,null


In [0]:
%sql
VACUUM f1_demo.drivers_merge

path
dbfs:/mnt/formula1dl612/demo/drivers_merge


In [0]:
%sql
SELECT * from f1_demo.drivers_merge TIMESTAMP AS OF '2024-03-16T23:27:41.000+0000';

driverID,dob,forename,surname,createdDate,updatedDate
1,1985-01-07,Lewis,Hamilton,2024-03-16,null
2,1977-05-10,Nick,Heidfeld,2024-03-16,null
3,1985-06-27,Nico,Rosberg,2024-03-16,null
4,1981-07-29,Fernando,Alonso,2024-03-16,null
5,1981-10-19,Heikki,Kovalainen,2024-03-16,null
6,1985-01-11,Kazuki,Nakajima,2024-03-16,null
7,1979-02-28,Sébastien,Bourdais,2024-03-16,null
8,1979-10-17,Kimi,Räikkönen,2024-03-16,null
9,1984-12-07,Robert,Kubica,2024-03-16,null
10,1982-03-18,Timo,Glock,2024-03-16,null


In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false;
VACUUM f1_demo.drivers_merge RETAIN 0 HOURS

path
dbfs:/mnt/formula1dl612/demo/drivers_merge


In [0]:
%sql
SELECT * from f1_demo.drivers_merge TIMESTAMP AS OF '2024-03-16T23:27:41.000+0000';

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 157.0 failed 4 times, most recent failure: Lost task 0.3 in stage 157.0 (TID 2123) (10.139.64.4 executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/mnt/formula1dl612/demo/drivers_merge/part-00000-d1b6711b-5896-451d-b4a2-339255b51e44-c000.snappy.parquet.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:654)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:623)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:747)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:456)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:451)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:1983)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:761)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:80)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:186)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:174)
	at org.apache.spark.scheduler.Task.$anonfun$run$4(Task.scala:137)
	at com.databricks.unity.UCSEphemeralState$Handle.runWith(UCSEphemeralState.scala:41)
	at com.databricks.unity.HandleImpl.runWith(UCSHandle.scala:99)
	at com.databricks.unity.HandleImpl.$anonfun$runWithAndClose$1(UCSHandle.scala:104)
	at scala.util.Using$.resource(Using.scala:269)
	at com.databricks.unity.HandleImpl.runWithAndClose(UCSHandle.scala:103)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:137)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:96)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:902)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1697)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:905)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:760)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.io.IOException: java.io.FileNotFoundException: Operation failed: "The specified path does not exist.", 404, GET, https://formula1dl612.dfs.core.windows.net/demo/drivers_merge/part-00000-d1b6711b-5896-451d-b4a2-339255b51e44-c000.snappy.parquet?timeout=90, P

In [0]:
%sql
SELECT * from f1_demo.drivers_merge

driverID,dob,forename,surname,createdDate,updatedDate
6,1985-01-11,KAZUKI,NAKAJIMA,2024-03-16,2024-03-16
7,1979-02-28,SÉBASTIEN,BOURDAIS,2024-03-16,2024-03-16
8,1979-10-17,KIMI,RÄIKKÖNEN,2024-03-16,2024-03-16
9,1984-12-07,ROBERT,KUBICA,2024-03-16,2024-03-16
10,1982-03-18,TIMO,GLOCK,2024-03-16,2024-03-16
11,1977-01-28,TAKUMA,SATO,2024-03-16,null
12,1985-07-25,NELSON,PIQUET JR.,2024-03-16,null
13,1981-04-25,FELIPE,MASSA,2024-03-16,null
14,1971-03-27,DAVID,COULTHARD,2024-03-16,null
15,1974-07-13,JARNO,TRULLI,2024-03-16,null


In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
7,2024-03-17T11:08:10.000+0000,2818554889652103,gilbaram20@outlook.com,VACUUM END,Map(status -> COMPLETED),null,List(4462865897687888),0124-093932-co5lwa5n,6,SnapshotIsolation,true,"Map(numDeletedFiles -> 2, numVacuumedDirectories -> 1)",null,Databricks-Runtime/11.3.x-scala2.12
6,2024-03-17T11:08:06.000+0000,2818554889652103,gilbaram20@outlook.com,VACUUM START,"Map(retentionCheckEnabled -> false, specifiedRetentionMillis -> 0, defaultRetentionMillis -> 604800000)",null,List(4462865897687888),0124-093932-co5lwa5n,5,SnapshotIsolation,true,"Map(numFilesToDelete -> 2, sizeOfDataToDelete -> 3987)",null,Databricks-Runtime/11.3.x-scala2.12
5,2024-03-17T11:06:11.000+0000,2818554889652103,gilbaram20@outlook.com,VACUUM END,Map(status -> COMPLETED),null,List(4462865897687888),0124-093932-co5lwa5n,4,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/11.3.x-scala2.12
4,2024-03-17T11:06:06.000+0000,2818554889652103,gilbaram20@outlook.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(4462865897687888),0124-093932-co5lwa5n,3,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/11.3.x-scala2.12
3,2024-03-16T23:52:27.000+0000,2818554889652103,gilbaram20@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#16346 as bigint) = driverId#16299L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(4462865897687888),0124-093932-co5lwa5n,2,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 2551, numTargetRowsInserted -> 5, scanTimeMs -> 1199, numTargetRowsUpdated -> 5, numOutputRows -> 10, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 1, rewriteTimeMs -> 1030)",null,Databricks-Runtime/11.3.x-scala2.12
2,2024-03-16T23:31:48.000+0000,2818554889652103,gilbaram20@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#15045 as bigint) = driverId#14971L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(4462865897687888),0124-093932-co5lwa5n,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 5, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, executionTimeMs -> 2990, numTargetRowsInserted -> 5, scanTimeMs -> 1502, numTargetRowsUpdated -> 5, numOutputRows -> 15, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 1, rewriteTimeMs -> 972)",null,Databricks-Runtime/11.3.x-scala2.12
1,2024-03-16T23:27:41.000+0000,2818554889652103,gilbaram20@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#13823 as bigint) = driverId#12836L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(4462865897687888),0124-093932-co5lwa5n,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 2680, numTargetRowsInserted -> 10, scanTimeMs -> 1524, numTargetRowsUpdated -> 0, numOutputRows -> 10, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 0, rewriteTimeMs -> 653)",null,Databricks-Runtime/11.3.x-scala2.12
0,2024-03-16T23:25:43.000+0000,2818554889652103,gilbaram20@outlook.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(4462865897687888),0124-093932-co5lwa5n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/11.3.x-scala2.12


In [0]:
%sql
DELETE from f1_demo.drivers_merge WHERE driverID = 1;

num_affected_rows
1


In [0]:
%sql
select * from f1_demo.drivers_merge version as of 3;


driverID,dob,forename,surname,createdDate,updatedDate
6,1985-01-11,KAZUKI,NAKAJIMA,2024-03-16,2024-03-16
7,1979-02-28,SÉBASTIEN,BOURDAIS,2024-03-16,2024-03-16
8,1979-10-17,KIMI,RÄIKKÖNEN,2024-03-16,2024-03-16
9,1984-12-07,ROBERT,KUBICA,2024-03-16,2024-03-16
10,1982-03-18,TIMO,GLOCK,2024-03-16,2024-03-16
11,1977-01-28,TAKUMA,SATO,2024-03-16,null
12,1985-07-25,NELSON,PIQUET JR.,2024-03-16,null
13,1981-04-25,FELIPE,MASSA,2024-03-16,null
14,1971-03-27,DAVID,COULTHARD,2024-03-16,null
15,1974-07-13,JARNO,TRULLI,2024-03-16,null


In [0]:
%sql
MERGE into f1_demo.drivers_merge tgt
USING f1_demo.drivers_merge VERSIOn AS Of 3 src
  On (tgt.driverId = src.driverId)
When not MATCHED THEN
  INSERT *  

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,0,0,1


In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
9,2024-03-17T11:14:48.000+0000,2818554889652103,gilbaram20@outlook.com,MERGE,"Map(predicate -> [""(driverId#6065 = driverId#6071)""], matchedPredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(4462865897687888),0124-093932-co5lwa5n,8,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 1899, numTargetRowsInserted -> 1, scanTimeMs -> 0, numTargetRowsUpdated -> 0, numOutputRows -> 1, numTargetChangeFilesAdded -> 0, numSourceRows -> 24, numTargetFilesRemoved -> 0, rewriteTimeMs -> 1838)",null,Databricks-Runtime/11.3.x-scala2.12
8,2024-03-17T11:11:56.000+0000,2818554889652103,gilbaram20@outlook.com,DELETE,"Map(predicate -> [""(driverID#5272 = 1)""])",null,List(4462865897687888),0124-093932-co5lwa5n,7,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 9, numAddedChangeFiles -> 0, executionTimeMs -> 2216, numDeletedRows -> 1, scanTimeMs -> 705, numAddedFiles -> 1, rewriteTimeMs -> 1505)",null,Databricks-Runtime/11.3.x-scala2.12
7,2024-03-17T11:08:10.000+0000,2818554889652103,gilbaram20@outlook.com,VACUUM END,Map(status -> COMPLETED),null,List(4462865897687888),0124-093932-co5lwa5n,6,SnapshotIsolation,true,"Map(numDeletedFiles -> 2, numVacuumedDirectories -> 1)",null,Databricks-Runtime/11.3.x-scala2.12
6,2024-03-17T11:08:06.000+0000,2818554889652103,gilbaram20@outlook.com,VACUUM START,"Map(retentionCheckEnabled -> false, specifiedRetentionMillis -> 0, defaultRetentionMillis -> 604800000)",null,List(4462865897687888),0124-093932-co5lwa5n,5,SnapshotIsolation,true,"Map(numFilesToDelete -> 2, sizeOfDataToDelete -> 3987)",null,Databricks-Runtime/11.3.x-scala2.12
5,2024-03-17T11:06:11.000+0000,2818554889652103,gilbaram20@outlook.com,VACUUM END,Map(status -> COMPLETED),null,List(4462865897687888),0124-093932-co5lwa5n,4,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/11.3.x-scala2.12
4,2024-03-17T11:06:06.000+0000,2818554889652103,gilbaram20@outlook.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(4462865897687888),0124-093932-co5lwa5n,3,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/11.3.x-scala2.12
3,2024-03-16T23:52:27.000+0000,2818554889652103,gilbaram20@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#16346 as bigint) = driverId#16299L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(4462865897687888),0124-093932-co5lwa5n,2,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 2551, numTargetRowsInserted -> 5, scanTimeMs -> 1199, numTargetRowsUpdated -> 5, numOutputRows -> 10, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 1, rewriteTimeMs -> 1030)",null,Databricks-Runtime/11.3.x-scala2.12
2,2024-03-16T23:31:48.000+0000,2818554889652103,gilbaram20@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#15045 as bigint) = driverId#14971L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(4462865897687888),0124-093932-co5lwa5n,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 5, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, executionTimeMs -> 2990, numTargetRowsInserted -> 5, scanTimeMs -> 1502, numTargetRowsUpdated -> 5, numOutputRows -> 15, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 1, rewriteTimeMs -> 972)",null,Databricks-Runtime/11.3.x-scala2.12
1,2024-03-16T23:27:41.000+0000,2818554889652103,gilbaram20@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#13823 as bigint) = driverId#12836L)""],

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge;

driverID,dob,forename,surname,createdDate,updatedDate
6,1985-01-11,KAZUKI,NAKAJIMA,2024-03-16,2024-03-16
7,1979-02-28,SÉBASTIEN,BOURDAIS,2024-03-16,2024-03-16
8,1979-10-17,KIMI,RÄIKKÖNEN,2024-03-16,2024-03-16
9,1984-12-07,ROBERT,KUBICA,2024-03-16,2024-03-16
10,1982-03-18,TIMO,GLOCK,2024-03-16,2024-03-16
11,1977-01-28,TAKUMA,SATO,2024-03-16,null
12,1985-07-25,NELSON,PIQUET JR.,2024-03-16,null
13,1981-04-25,FELIPE,MASSA,2024-03-16,null
14,1971-03-27,DAVID,COULTHARD,2024-03-16,null
15,1974-07-13,JARNO,TRULLI,2024-03-16,null


Transaction Logs

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_txn (
driverID INT,
dob DATE,
forename STRING,
surname STRING,
createdDate DATE,
updatedDate DATE
)
USING DELTA

In [0]:
%sql
DESC HISTORY f1_demo.drivers_txn;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2024-03-17T11:19:08.000+0000,2818554889652103,gilbaram20@outlook.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(4462865897687888),0124-093932-co5lwa5n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/11.3.x-scala2.12


In [0]:
# f1_demo.drivers_txn/_delta_log

# {"commitInfo":{"timestamp":1710674348661,"userId":"2818554889652103","userName":"gilbaram20@outlook.com","operation":"CREATE TABLE","operationParameters":{"isManaged":"true","description":null,"partitionBy":"[]","properties":"{}"},"notebook":{"notebookId":"4462865897687888"},"clusterId":"0124-093932-co5lwa5n","isolationLevel":"WriteSerializable","isBlindAppend":true,"operationMetrics":{},"engineInfo":"Databricks-Runtime/11.3.x-scala2.12","txnId":"b3562a70-8dbf-47c5-a540-2338a047a2ee"}}
# {"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
# {"metaData":{"id":"4b6e8be9-4d61-478d-8c19-1e0437c53947","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"driverID\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"dob\",\"type\":\"date\",\"nullable\":true,\"metadata\":{}},{\"name\":\"forename\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"surname\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"createdDate\",\"type\":\"date\",\"nullable\":true,\"metadata\":{}},{\"name\":\"updatedDate\",\"type\":\"date\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1710674348575}}


In [0]:
%sql
insert into f1_demo.drivers_txn
select * from f1_demo.drivers_merge
Where driverId = 1;

num_affected_rows,num_inserted_rows
1,1


In [0]:
# {"commitInfo":{"timestamp":1710674669177,"userId":"2818554889652103","userName":"gilbaram20@outlook.com","operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"notebook":{"notebookId":"4462865897687888"},"clusterId":"0124-093932-co5lwa5n","readVersion":0,"isolationLevel":"WriteSerializable","isBlindAppend":true,"operationMetrics":{"numFiles":"1","numOutputRows":"1","numOutputBytes":"1836"},"engineInfo":"Databricks-Runtime/11.3.x-scala2.12","txnId":"bb65cefe-4e0b-4c56-9cd6-92e1902e5852"}}
# {"add":{"path":"part-00000-4f51e96e-404c-40a6-99c2-b1dab8e5fa27-c000.snappy.parquet","partitionValues":{},"size":1836,"modificationTime":1710674669000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"driverID\":1,\"dob\":\"1985-01-07\",\"forename\":\"LEWIS\",\"surname\":\"HAMILTON\",\"createdDate\":\"2024-03-16\",\"updatedDate\":\"2024-03-16\"},\"maxValues\":{\"driverID\":1,\"dob\":\"1985-01-07\",\"forename\":\"LEWIS\",\"surname\":\"HAMILTON\",\"createdDate\":\"2024-03-16\",\"updatedDate\":\"2024-03-16\"},\"nullCount\":{\"driverID\":0,\"dob\":0,\"forename\":0,\"surname\":0,\"createdDate\":0,\"updatedDate\":0}}","tags":{"INSERTION_TIME":"1710674669000000","MIN_INSERTION_TIME":"1710674669000000","MAX_INSERTION_TIME":"1710674669000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}


In [0]:
%sql
DESC HISTORY f1_demo.drivers_txn

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2024-03-17T11:24:29.000+0000,2818554889652103,gilbaram20@outlook.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4462865897687888),0124-093932-co5lwa5n,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1836)",null,Databricks-Runtime/11.3.x-scala2.12
0,2024-03-17T11:19:08.000+0000,2818554889652103,gilbaram20@outlook.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(4462865897687888),0124-093932-co5lwa5n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/11.3.x-scala2.12


In [0]:
%sql
insert into f1_demo.drivers_txn
select * from f1_demo.drivers_merge
Where driverId = 2;

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
DELETE FROM f1_demo.drivers_txn
WHERE driverId = 1;

num_affected_rows
1


In [0]:
for driver_id in range(3,20):
    spark.sql(f"""insert into f1_demo.drivers_txn
                    select * from f1_demo.drivers_merge
                    Where driverId = {driver_id}""")

In [0]:
%sql
INSERT Into f1_demo.drivers_txn
SELECT * From f1_demo.drivers_merge;

num_affected_rows,num_inserted_rows
20,20


Convert Parquet to Delta

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_convert_to_delta (
driverID INT,
dob DATE,
forename STRING,
surname STRING,
createdDate DATE,
updatedDate DATE
)
USING PARQUET

In [0]:
%sql
insert into f1_demo.drivers_convert_to_delta
select * from f1_demo.drivers_merge

In [0]:
%sql
convert TO Delta f1_demo.drivers_convert_to_delta

In [0]:
df = spark.table("f1_demo.drivers_convert_to_delta")

In [0]:
df.write.format("parquet").mode("overwrite").save("/mnt/formula1dl612/demo/drivers_convert_to-delta_new")

In [0]:
%sql
CONVERT TO DELTA parquet.`/mnt/formula1dl612/demo/drivers_convert_to-delta_new`